In [ ]:
df_anxiety = pd.read_csv('Anxiety.csv')
df_ptsd = pd.read_csv('PTSD.csv')

In [ ]:
df_anxiety = df_anxiety.drop(columns = 'Unnamed: 0')
df_ptsd.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
df = pd.concat([df_ptsd,df_anxiety]).head(2)
df.reset_index(inplace = True)
df.drop(columns = 'index', inplace = True)

In [ ]:
# define function to remove punctuation and remove \n
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    punctuationfree = ' '.join([sentence for sentence in punctuationfree.split('\n') if sentence != ''])
    return punctuationfree

columns = ['selftext', 'title']

for col in columns:
    new_col = str('clean_' + col)
    df[new_col] = df[col].apply(lambda x: remove_punctuation(x))   

In [ ]:
# changing all to lower caps 

df_ptsd['clean_msg']= df_ptsd['clean_msg'].apply(lambda x: x.lower())
df_anxiety['clean_msg']= df_anxiety['clean_msg'].apply(lambda x: x.lower())

In [ ]:
# split into single words 
df_ptsd['msg_tokenied']= df_ptsd['clean_msg'].apply(lambda x: x.split())
df_anxiety['msg_tokenied']= df_anxiety['clean_msg'].apply(lambda x: x.split())

In [ ]:
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

In [ ]:
#applying the function
df_anxiety['no_stopwords']= df_anxiety['msg_tokenied'].apply(lambda x:remove_stopwords(x))
df_ptsd['no_stopwords']= df_ptsd['msg_tokenied'].apply(lambda x:remove_stopwords(x))

In [ ]:
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text


df_anxiety['msg_lemmatized']=df_anxiety['no_stopwords'].apply(lambda x:lemmatizer(x))
df_ptsd['msg_lemmatized']=df_ptsd['no_stopwords'].apply(lambda x:lemmatizer(x))